**Amazon_news**

In [ ]:
# gdelt_amazon_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- USER CONFIG ----------
QUERY = 'AMAZON'                    # company keyword
START_DATE = datetime(2019, 1, 1)   # from 2019
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "amazon_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10            # we will later keep only 1 row/day
REQUEST_SLEEP = 0.6
# ----------------------------------

# requests session with retries
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-amazon-fetch-v4/1.0 (+https://example)"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    """
    Build GDELT URL for a specific date window.
    Here we enforce English only via sourcelang:english
    """
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    url = (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )
    return url

def fetch_csv_text(url, timeout=30):
    """Download a CSV string from GDELT"""
    try:
        r = session.get(url, timeout=timeout)
        if r.status_code != 200:
            return None
        text = r.text
        if not text.strip():
            return None
        return text
    except Exception:
        return None

def parse_csv_to_df(text):
    """Convert GDELT CSV response to pandas DataFrame"""
    try:
        df = pd.read_csv(StringIO(text), on_bad_lines='skip')
        return df
    except Exception:
        return None

def get_one_article_for_day(day_dt):
    """
    Fetch up to MAXRECORDS_PER_DAY articles for a single day,
    filter to English, and return AT MOST ONE row (or None if no news).
    """
    # full-day window [00:00:00, 23:59:59] for that calendar day
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv_to_df(text)
    if df is None or df.empty:
        return None

    # keep English-only (extra safety, though query already has sourcelang:english)
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # ---- choose exactly one article for that day ----
    # If there is a date column, we can sort by it; otherwise just take the first row.
    # Common candidates: 'SQLDATE', 'DATE', or 'DocumentIdentifier' ordering.
    if 'SQLDATE' in df.columns:
        # SQLDATE is usually yyyymmdd
        df = df.sort_values('SQLDATE', ascending=True)
    elif 'Date' in df.columns:
        df = df.sort_values('Date', ascending=True)

    one_row = df.head(1).copy()

    # Add the query date explicitly so it's easy to join with price data later
    one_row['QueryDate'] = day_dt.date()

    return one_row

def run_daily_tradingday_download():
    """
    Loop over every day from START_DATE to END_DATE.
    - Only Monday–Friday (approx trading days)
    - For each such day, keep at most 1 article
    - Save all days into a single CSV at the end
    """
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    # approximate number of days in range just for progress bar
    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="days (calendar)")

    while current <= END_DATE:
        # skip weekends (Sat=5, Sun=6)
        if current.weekday() < 5:
            # approx trading day -> try to get exactly 1 news row
            try:
                row_df = get_one_article_for_day(current)
            except Exception as e:
                print("Error for day", current.date(), ":", e)
                row_df = None

            if row_df is not None and not row_df.empty:
                all_rows.append(row_df)

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No articles found in the entire range.")
        return

    final_df = pd.concat(all_rows, ignore_index=True)

    # Just in case, enforce max 1 row per QueryDate by grouping and taking first
    final_df = (
        final_df.sort_values('QueryDate')
                .groupby('QueryDate', as_index=False)
                .first()
    )

    final_df.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Finished.")
    print("Total trading days with at least 1 article:", len(final_df))
    if os.path.exists(OUT_CSV):
        print("Saved CSV:", OUT_CSV, "Size (MB):", os.path.getsize(OUT_CSV) / (1024*1024))

if __name__ == "__main__":
    print("Starting GDELT daily trading-day download (1 news per day) for AMAZON...")
    run_daily_tradingday_download()


Starting GDELT daily trading-day download (1 news per day) for AMAZON...


days (calendar): 100%|██████████| 2192/2192 [37:15<00:00,  1.02s/it]

✅ Finished.
Total trading days with at least 1 article: 1564
Saved CSV: amazon_news_2019_2024_1news_per_tradingday.csv Size (MB): 0.41716480255126953


In [ ]:
from google.colab import files
files.download("amazon_news_2019_2024_1news_per_tradingday.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Apple_News**

In [ ]:
# gdelt_apple_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- USER CONFIG ----------
QUERY = 'APPLE'                    # company keyword
START_DATE = datetime(2019, 1, 1)  # from 2019
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "apple_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10           # we later keep only 1 row/day
REQUEST_SLEEP = 0.6
# ----------------------------------

# requests session with retries
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-apple-fetch-v1/1.0 (+https://example)"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    """
    Build GDELT URL for a specific date window.
    Enforces English via sourcelang:english
    """
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    url = (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )
    return url

def fetch_csv_text(url, timeout=30):
    """Download a CSV string from GDELT"""
    try:
        r = session.get(url, timeout=timeout)
        if r.status_code != 200:
            return None
        text = r.text
        if not text.strip():
            return None
        return text
    except Exception:
        return None

def parse_csv_to_df(text):
    """Convert GDELT CSV response to pandas DataFrame"""
    try:
        df = pd.read_csv(StringIO(text), on_bad_lines='skip')
        return df
    except Exception:
        return None

def get_one_article_for_day(day_dt):
    """
    Fetch up to MAXRECORDS_PER_DAY articles for a single day,
    filter to English, and return AT MOST ONE row.
    """
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv_to_df(text)
    if df is None or df.empty:
        return None

    # keep English-only
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # sort by date if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE', ascending=True)
    elif 'Date' in df.columns:
        df = df.sort_values('Date', ascending=True)

    # pick first row
    one_row = df.head(1).copy()

    # Add QueryDate for merging with price data later
    one_row['QueryDate'] = day_dt.date()

    return one_row

def run_daily_tradingday_download():
    """
    Loop through every trading day (Mon–Fri) between START_DATE and END_DATE.
    Save at most 1 news article per day.
    """
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="days (calendar)")

    while current <= END_DATE:

        # trading day = Monday–Friday
        if current.weekday() < 5:
            try:
                row_df = get_one_article_for_day(current)
            except Exception as e:
                print("Error for day", current.date(), ":", e)
                row_df = None

            if row_df is not None and not row_df.empty:
                all_rows.append(row_df)

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No articles found in the entire range.")
        return

    final_df = pd.concat(all_rows, ignore_index=True)

    # ensure 1 row per QueryDate
    final_df = (
        final_df.sort_values('QueryDate')
                .groupby('QueryDate', as_index=False)
                .first()
    )

    final_df.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Finished.")
    print("Total trading days with at least 1 article:", len(final_df))
    if os.path.exists(OUT_CSV):
        print("Saved CSV:", OUT_CSV, "Size (MB):", os.path.getsize(OUT_CSV) / (1024*1024))

if __name__ == "__main__":
    print("Starting GDELT daily trading-day download (1 news per day) for APPLE...")
    run_daily_tradingday_download()


Starting GDELT daily trading-day download (1 news per day) for APPLE...


days (calendar): 100%|██████████| 2192/2192 [36:22<00:00,  1.00it/s]


✅ Finished.
Total trading days with at least 1 article: 1564
Saved CSV: apple_news_2019_2024_1news_per_tradingday.csv Size (MB): 0.40357017517089844


In [ ]:
from google.colab import files
files.download("apple_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Google**

In [ ]:
# gdelt_google_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = 'GOOGLE OR ALPHABET'
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "google_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-google-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        if current.weekday() < 5:  # Monday–Friday
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found!")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False)

    print("✅ Done!")
    print("Total trading days with news:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting GOOGLE daily news download…")
    run_download()


Starting GOOGLE daily news download…


Processing days: 100%|██████████| 2192/2192 [20:40<00:00,  1.77it/s]

No news found!


In [ ]:
# gdelt_google_1news_per_tradingday_2019_2024_fixed.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = 'GOOGLE'   # simpler, like your AMAZON query
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "google_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-google-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    # same trick as before: enforce English via sourcelang:english
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    # keep English-only if column exists
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # sort by some date-like column if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    # pick just 1 article
    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        # Monday–Friday only
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – check query or try a closer date range.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting GOOGLE daily news download…")
    run_download()


Starting GOOGLE daily news download…


Processing days: 100%|██████████| 2192/2192 [38:45<00:00,  1.06s/it]


✅ Done!
Total trading days with at least 1 article: 1564
Saved to: google_news_2019_2024_1news_per_tradingday.csv


In [ ]:
from google.colab import files
files.download("google_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Nvidia**

In [ ]:
# gdelt_nvidia_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = 'NVIDIA'
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "nvidia_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-nvidia-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    # enforce English sources
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    # keep English-only if column exists
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # sort by some date-like column if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    # pick just 1 article
    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        # Monday–Friday only
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – check query or date range.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting NVIDIA daily news download…")
    run_download()


Starting NVIDIA daily news download…


Processing days: 100%|██████████| 2192/2192 [33:38<00:00,  1.09it/s]


✅ Done!
Total trading days with at least 1 article: 1564
Saved to: nvidia_news_2019_2024_1news_per_tradingday.csv


In [ ]:
from google.colab import files
files.download("nvidia_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# gdelt_microsoft_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = 'MICROSOFT'
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "microsoft_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-microsoft-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    # enforce English sources
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    # keep English-only if column exists
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # sort by some date-like column if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    # pick just 1 article
    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        # Monday–Friday only
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – check query or date range.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting MICROSOFT daily news download…")
    run_download()


Starting MICROSOFT daily news download…


Processing days: 100%|██████████| 2192/2192 [35:03<00:00,  1.04it/s]

✅ Done!
Total trading days with at least 1 article: 1564
Saved to: microsoft_news_2019_2024_1news_per_tradingday.csv


In [ ]:
from google.colab import files
files.download("microsoft_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**S&P 500**

In [ ]:
# gdelt_sp500_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
# Phrase search for the index itself
QUERY = '"S&P 500"'
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "sp500_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-sp500-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    # enforce English sources
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    # keep English-only if column exists
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # sort by some date-like column if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    # pick just 1 article
    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        # Monday–Friday only (approx trading days)
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – try adjusting QUERY or date range.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting S&P 500 daily news download…")
    run_download()


Starting S&P 500 daily news download…


Processing days: 100%|██████████| 2192/2192 [36:47<00:00,  1.01s/it]

✅ Done!
Total trading days with at least 1 article: 1564
Saved to: sp500_news_2019_2024_1news_per_tradingday.csv


In [ ]:
from google.colab import files
files.download("sp500_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**AstraZeneca PLO**

In [ ]:
# gdelt_astrazeneca_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = 'ASTRAZENECA'
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "astrazeneca_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-astrazeneca-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    # enforce English sources
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    # keep English-only if column exists
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # sort by some date-like column if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    # pick just 1 article
    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        # Monday–Friday only
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – check query or date range.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting ASTRAZENECA daily news download…")
    run_download()


Starting ASTRAZENECA daily news download…


Processing days: 100%|██████████| 2192/2192 [36:33<00:00,  1.00s/it]

✅ Done!
Total trading days with at least 1 article: 1564
Saved to: astrazeneca_news_2019_2024_1news_per_tradingday.csv


In [ ]:
from google.colab import files
files.download("astrazeneca_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**HSBC**

In [ ]:
# gdelt_hsbc_holdings_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
# Slightly focused query for the group
QUERY = '"HSBC HOLDINGS"'
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "hsbc_holdings_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-hsbc-holdings-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    # enforce English sources
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    # keep English-only if column exists
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # sort by some date-like column if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    # pick just 1 article
    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        # Monday–Friday only (approx trading days)
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – try adjusting QUERY or date range.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting HSBC HOLDINGS daily news download…")
    run_download()


Starting HSBC HOLDINGS daily news download…


Processing days: 100%|██████████| 2192/2192 [42:57<00:00,  1.18s/it]

✅ Done!
Total trading days with at least 1 article: 1552
Saved to: hsbc_holdings_news_2019_2024_1news_per_tradingday.csv


In [ ]:
from google.colab import files
files.download("hsbc_holdings_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Linde PLC**

In [ ]:
# gdelt_linde_plc_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = '"LINDE PLC"'   # you can change to 'LINDE' if this is too narrow
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "linde_plc_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-linde-plc-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    # enforce English sources
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    # keep English-only if column exists
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # sort by some date-like column if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    # pick just 1 article
    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        # Monday–Friday only (approx trading days)
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – you might try QUERY = 'LINDE' instead.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting LINDE PLC daily news download…")
    run_download()


Starting LINDE PLC daily news download…


Processing days: 100%|██████████| 2192/2192 [52:23<00:00,  1.43s/it]

✅ Done!
Total trading days with at least 1 article: 1285
Saved to: linde_plc_news_2019_2024_1news_per_tradingday.csv


In [ ]:
from google.colab import files
files.download("linde_plc_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Shell PLC**

In [ ]:
# gdelt_shell_plc_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = '"SHELL PLC"'   # you can change to 'SHELL' or '"ROYAL DUTCH SHELL"' if needed
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "shell_plc_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-shell-plc-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    # enforce English sources
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    # keep English-only if column exists
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # sort by some date-like column if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    # pick just 1 article
    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        # Monday–Friday only (approx trading days)
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – you might try QUERY = 'SHELL' or '\"ROYAL DUTCH SHELL\"' instead.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting SHELL PLC daily news download…")
    run_download()


Starting SHELL PLC daily news download…


Processing days: 100%|██████████| 2192/2192 [50:15<00:00,  1.38s/it]


✅ Done!
Total trading days with at least 1 article: 1549
Saved to: shell_plc_news_2019_2024_1news_per_tradingday.csv


In [ ]:
from google.colab import files
files.download("shell_plc_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Unilever PLO**

In [ ]:
# gdelt_unilever_plc_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = '"UNILEVER PLC"'   # if too narrow, change to 'UNILEVER'
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "unilever_plc_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-unilever-plc-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    # enforce English sources
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    # keep English-only if column exists
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # sort by some date-like column if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    # pick just 1 article
    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        # Monday–Friday only (approx trading days)
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – you might try QUERY = 'UNILEVER' instead.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting UNILEVER PLC daily news download…")
    run_download()


Starting UNILEVER PLC daily news download…


Processing days: 100%|██████████| 2192/2192 [39:51<00:00,  1.09s/it]


✅ Done!
Total trading days with at least 1 article: 1450
Saved to: unilever_plc_news_2019_2024_1news_per_tradingday.csv


In [ ]:
from google.colab import files
files.download("unilever_plc_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
companies = [
    ("AMAZON", "amazon_news_...csv"),
    ("APPLE", "apple_news_...csv"),
    ...
    ('"UNILEVER PLC"', "unilever_plc_news_...csv"),
]


<>:4: SyntaxWarning: 'ellipsis' object is not callable; perhaps you missed a comma?
<>:4: SyntaxWarning: 'ellipsis' object is not callable; perhaps you missed a comma?
/tmp/ipython-input-3594943482.py:4: SyntaxWarning: 'ellipsis' object is not callable; perhaps you missed a comma?
  ...


TypeError: 'ellipsis' object is not callable

**FTSE100**

In [ ]:
# gdelt_ftse_100_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = '"FTSE 100"'
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "ftse_100_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-ftse-100-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    # enforce English sources
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    # keep English-only if column exists
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # sort by some date-like column if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    # pick just 1 article
    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        # Monday–Friday only (approx trading days)
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – check query or date range.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting FTSE 100 daily news download…")
    run_download()


Starting FTSE 100 daily news download…


Processing days: 100%|██████████| 2192/2192 [51:57<00:00,  1.42s/it]

✅ Done!
Total trading days with at least 1 article: 1564
Saved to: ftse_100_news_2019_2024_1news_per_tradingday.csv


In [ ]:
from google.colab import files
files.download("ftse_100_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# gdelt_toyota_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = 'TOYOTA MOTOR CORPORATION'
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "toyota_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-toyota-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except Exception:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except Exception:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – check the QUERY or date range.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting TOYOTA MOTOR CORPORATION daily news download…")
    run_download()


Starting TOYOTA MOTOR CORPORATION daily news download…


Processing days: 100%|██████████| 2192/2192 [41:43<00:00,  1.14s/it]


✅ Done!
Total trading days with at least 1 article: 1542
Saved to: toyota_news_2019_2024_1news_per_tradingday.csv


In [ ]:
from google.colab import files
files.download("toyota_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Mitsubishi**


In [ ]:
# gdelt_mufg_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = '"Mitsubishi UFJ Financial Group"'
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "mufg_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-mufg-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    # enforce English sources
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    # keep English-only if column exists
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # sort by some date-like column if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    # pick just 1 article
    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        # Monday–Friday only (approx trading days)
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – check query or date range.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting Mitsubishi UFJ Financial Group (MUFG) daily news download…")
    run_download()


Starting Mitsubishi UFJ Financial Group (MUFG) daily news download…


Processing days: 100%|██████████| 2192/2192 [42:22<00:00,  1.16s/it]


✅ Done!
Total trading days with at least 1 article: 1330
Saved to: mufg_news_2019_2024_1news_per_tradingday.csv


**Sony Group**

In [ ]:
# gdelt_sony_group_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = '"SONY GROUP CORPORATION"'   # you can change to 'SONY' if too narrow
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "sony_group_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-sony-group-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        return r.text if r.text.strip() else None
    except:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – check QUERY or date range.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf‑8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting SONY GROUP CORPORATION daily news download…")
    run_download()


Starting SONY GROUP CORPORATION daily news download…


Processing days: 100%|██████████| 2192/2192 [34:41<00:00,  1.05it/s]

✅ Done!
Total trading days with at least 1 article: 603
Saved to: sony_group_news_2019_2024_1news_per_tradingday.csv


In [ ]:
from google.colab import files
files.download("sony_group_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**HITACHI**

In [ ]:
# gdelt_hitachi_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = '"HITACHI LTD."'   # you can also try 'HITACHI' if needed
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV = "hitachi_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-hitachi-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        text = r.text
        if not text.strip():
            return None
        return text
    except Exception:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found – you may need to adjust QUERY")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf‑8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting HITACHI LTD daily news download…")
    run_download()


Starting HITACHI LTD daily news download…


Processing days: 100%|██████████| 2192/2192 [45:03<00:00,  1.23s/it]


✅ Done!
Total trading days with at least 1 article: 1512
Saved to: hitachi_news_2019_2024_1news_per_tradingday.csv


In [ ]:
from google.colab import files
files.download("hitachi_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Nikkei**

In [18]:
# gdelt_nikkei_1news_per_tradingday_2019_2024_FIXED.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
# Broader query than just "Nikkei 225"
QUERY = 'NIKKEI'
START_DATE = datetime(2019, 1, 1)
END_DATE   = datetime(2024, 12, 31)
OUT_CSV    = "nikkei_news_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP      = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-nikkei-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    # still constrain to English sources via sourcelang:english
    q = quote_plus(f'{query} sourcelang:english')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        text = r.text
        if not text.strip():
            return None
        return text
    except Exception:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    # keep English-only if column exists
    if 'DocumentLanguage' in df.columns:
        df = df[df['DocumentLanguage'].astype(str).str.lower() == 'english']

    if df.empty:
        return None

    # sort by some date-like column if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    # pick just 1 article
    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        # Monday–Friday only (approx trading days)
        if current.weekday() < 5:
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error for day {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found at all – even with broader query. Then GDELT coverage for this term may be very sparse.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf-8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting Nikkei daily news download (broader query: NIKKEI)…")
    run_download()


Starting Nikkei daily news download (broader query: NIKKEI)…


Processing days: 100%|██████████| 2192/2192 [47:40<00:00,  1.31s/it]

✅ Done!
Total trading days with at least 1 article: 1564
Saved to: nikkei_news_2019_2024_1news_per_tradingday.csv


In [19]:
from google.colab import files
files.download("nikkei_news_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Nintendo**

In [25]:
# gdelt_nintendo_no_lang_filter_1news_per_tradingday_2019_2024.py
import os
import time
from io import StringIO
from datetime import datetime, timedelta
from urllib.parse import quote_plus

import requests
import pandas as pd
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------- CONFIG ----------
QUERY = '"Nintendo"'  # Broaden query
START_DATE = datetime(2019, 1, 1)  # Start with Jan 2019
END_DATE   = datetime(2024, 12, 31)  # End with Dec 2024
OUT_CSV = "nintendo_news_all_lang_2019_2024_1news_per_tradingday.csv"
MAXRECORDS_PER_DAY = 10
REQUEST_SLEEP = 0.6
# ----------------------------

session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("http://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "gdelt-nintendo-fetch/1.0"})

def make_url(query, start_dt, end_dt, maxrecords=MAXRECORDS_PER_DAY, fmt="csv"):
    # No language filter, so we fetch all languages
    q = quote_plus(f'{query}')
    start_s = start_dt.strftime("%Y%m%d") + "000000"
    end_s   = end_dt.strftime("%Y%m%d") + "235959"
    return (
        f"http://api.gdeltproject.org/api/v2/doc/doc"
        f"?query={q}"
        f"&mode=artlist"
        f"&maxrecords={maxrecords}"
        f"&format={fmt}"
        f"&startdatetime={start_s}"
        f"&enddatetime={end_s}"
    )

def fetch_csv_text(url):
    try:
        r = session.get(url, timeout=30)
        if r.status_code != 200:
            return None
        text = r.text
        if not text.strip():
            return None
        return text
    except Exception:
        return None

def parse_csv(text):
    try:
        return pd.read_csv(StringIO(text), on_bad_lines='skip')
    except Exception:
        return None

def get_one_news(day_dt):
    url = make_url(QUERY, day_dt, day_dt)
    text = fetch_csv_text(url)
    time.sleep(REQUEST_SLEEP)

    if text is None:
        return None

    df = parse_csv(text)
    if df is None or df.empty:
        return None

    # Sort by some date-like column if available
    if 'SQLDATE' in df.columns:
        df = df.sort_values('SQLDATE')
    elif 'Date' in df.columns:
        df = df.sort_values('Date')

    row = df.head(1).copy()
    row['QueryDate'] = day_dt.date()
    return row

def run_download():
    if os.path.exists(OUT_CSV):
        os.remove(OUT_CSV)

    all_rows = []
    current = START_DATE

    total_days = (END_DATE - START_DATE).days + 1
    pbar = tqdm(total=total_days, desc="Processing days")

    while current <= END_DATE:
        if current.weekday() < 5:  # Monday–Friday
            try:
                r = get_one_news(current)
                if r is not None and not r.empty:
                    all_rows.append(r)
            except Exception as e:
                print(f"Error for day {current.date()}: {e}")

        current += timedelta(days=1)
        pbar.update(1)

    pbar.close()

    if not all_rows:
        print("No news found — please check query or date range.")
        return

    final = pd.concat(all_rows, ignore_index=True)
    final = final.sort_values('QueryDate').groupby('QueryDate', as_index=False).first()
    final.to_csv(OUT_CSV, index=False, encoding="utf‑8")

    print("✅ Done!")
    print("Total trading days with at least 1 article:", len(final))
    print("Saved to:", OUT_CSV)

if __name__ == "__main__":
    print("Starting NINTENDO daily news download (no language filter)…")
    run_download()


Starting NINTENDO daily news download (no language filter)…


Processing days: 100%|██████████| 2192/2192 [1:02:25<00:00,  1.71s/it]


✅ Done!
Total trading days with at least 1 article: 1561
Saved to: nintendo_news_all_lang_2019_2024_1news_per_tradingday.csv


In [26]:
from google.colab import files
files.download("nintendo_news_all_lang_2019_2024_1news_per_tradingday.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>